In [ ]:
# Crawler for conference speakers images
# uses list of conferences from wikipedia
# the resut is images from conference speakers stored in ./data

import urllib.request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import requests
import re
import random

conf_url_list =[]
unique_conf_set = set()
speakerspage_set = set()            
imgurls_set = set()
checkforspeakers_set = set()

# convert the html page into beautiful soup.
# input: templink: url source and 
# return: parsed html of the link provided
def getSoup(templink):
    try:
        temp_link_request = requests.get(templink)
        return BeautifulSoup(temp_link_request.content, 'html.parser')
    except:
        print("something went wrong please try again")
        return BeautifulSoup("", 'html.parser')

# downloads images from one url
# input list of html pages. the function loops throu the loop and downloads images
def downloadImages(speakerspages):
    for ind, speaker_page in enumerate(speakerspages):
        img_dic = { }
        speaker_soup = getSoup(speaker_page)           
        for img in speaker_soup.findAll('img'):
            imgurl = urljoin(speaker_page, img.get('src'))
            if (imgurl not in imgurls_set):
                file_type = imgurl[-4:]
                if (img.get("alt") != None) and (img.get("alt") != ""):
                    value= img.get("alt") + file_type
                else:
                    value= str(random.randint(1,999999999)) + file_type
                img_dic[imgurl] = value   
                imgurls_set.add(imgurl)
        counter = 0;
        for key, value in img_dic.items():
            try:
                urllib.request.urlretrieve(key, '../data/' + value)
                counter = counter + 1
                print("\t \t \t Downloading images: ", counter,"/", len(img_dic))
            except:
                print("something went wrong please try again")  

# function which searches for speakers pages where speakers might be announced
def searchSpeakers(templist):
    for i, conflink in enumerate(templist):
        speakers_list = []
        try:
            print("\t (", i+1,"/", len(templist),") ", conflink)
            confsoup = getSoup(conflink)
            speakers_list = confsoup.find_all("a", string = re.compile("Speakers"))
            speakers_list += confsoup.find_all("a", string = re.compile("Keynotes"))
        except OSError as err:
            print("ERROR", format(err))  
        print("\t \t FOUND ", len(speakers_list), " speakers page(s) ")
        for j, speakerpage in enumerate(speakers_list):
            if(hasattr(speakerpage, 'href')):
                speakerpage = urljoin(conflink,  speakerpage.get('href'))
                speakers_list[j] = speakerpage
                if(speakerpage not in speakerspage_set):
                    speakerspage_set.add(speakerpage)    
        downloadImages(speakers_list)

# makes absoulte link from base and relative part
# inputs: base: the domain, temp_list: relative urls of that domain
#return: the inputed temp_list with absolute links
def getHrefs(base, temp_list):
    for index, temp_link in enumerate(temp_list):
        absolute_link = urljoin(base,  temp_link.get('href'))
        if(absolute_link not in checkforspeakers_set):
            temp_list[index] = absolute_link
            checkforspeakers_set.add(absolute_link)
    return temp_list 

# crawls the wikipedia list of conferences and gets conferences links from two cs area
response = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_computer_science_conferences')
html = response.read().decode('utf-8')

first_topic = html[html.index('id="Languages_and_software"'):]
second_topic = html[html.index('id="Artificial_intelligence"'):]

conferences_list = first_topic[first_topic.find('<ul>'): first_topic.find('<h2>')]
conferences_list += second_topic[second_topic.find('<ul>'): second_topic.find('<h2>')]

soup = BeautifulSoup(conferences_list, 'html.parser')
conferences = [tag['href'] for tag in soup.select('li a[href]')]

# printing all the conferences which will be crawled
print("Conferneces:")
counter = 0
for conference in conferences:
    conf_wikipage = urljoin('https://en.wikipedia.org', conference)
    conf_wikipage_soup = getSoup(conf_wikipage)
    
    external_links = conf_wikipage_soup.find("span", id="External_links")
    if(hasattr(external_links, 'find_parent("h2")')):
        counter = counter + 1
        confurl = external_links.find_parent("h2").find_next_sibling("ul").find('a').get('href')
        conf_url_list.append(confurl)
        print(counter, ":", confurl)
        
print("\n *********************** \n")
print("SEARCHING FOR SPEAKERS PAGES")
print("\n *********************** \n")
searchSpeakers(conf_url_list)

print("\n *********************** \n")
print("SEARCHING FOR PRVIOUS YEARS CONFERENCES")
print("\n *********************** \n")

# looping throu each of the conferences to search for previous years conferences
for i, link in enumerate(conf_url_list):
    conf_soup = getSoup(link)            
    conferences_list = conf_soup.find_all("a", string = re.compile("Conference"))
    print("(", i+1,"/", len(conf_url_list),") ", link," - has ", len(conferences_list), "pages with conferences list")
    
    for index, conference_link in enumerate(conferences_list):
        conference_url = urljoin(link, conference_link.get('href'))
        if(conference_url not in unique_conf_set):
            try:
                print(index+1,"> Searching speakers . . . .")
                conf_list_soup = getSoup(conference_url)
                prev_year_conf = conf_list_soup.find_all("a")
                prevyearconf = getHrefs(link, prev_year_conf)
                speakers_page = searchSpeakers(prevyearconf) #contains all page with speaker title
                unique_conf_set.add(conference_url)
            except OSError as err:
                   print("ERROR", format(err))                
    print('\n\n')
        

Conferneces:
1 : http://www.etaps.org/
2 : http://www.ecoop.org/
3 : http://www.etaps.org/index.php/about/conferences
4 : http://research.ihost.com/hopl/HOPL-III.html
5 : http://www.cs.luc.edu/icfp/
6 : http://www.cs.nmsu.edu/ALP/conferences/
7 : http://www.sigplan.org/ismm.htm
8 : http://www.splashcon.org
9 : http://www.sigplan.org/Conferences/POPL/Main
10 : http://www.sigplan.org/Conferences/PLDI/Main
11 : http://www.ase-conferences.org/
12 : http://www.icse-conferences.org
13 : http://www.isase.org/pastconferences.htm
14 : http://www.omg.org/mda/mda_files/Model-Driven_Architecture.pdf
15 : http://cavconference.org
16 : http://www.aaai.org/
17 : http://www.aamas-conference.org/
18 : https://www.eurai.org/activities/ECAI_conferences
19 : http://www.ecmlpkdd.org
20 : http://icml.cc
21 : http://www.ijcai.org/
22 : http://nips.cc/
23 : http://www.informatik.uni-trier.de/~ley/db/conf/ruleml/
24 : https://ea2017.inria.fr//index.php
25 : http://www.wcci2016.org/
26 : http://sig.sigevo.org/i

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


	 	 FOUND  0  speakers page(s) 
	 ( 15 / 39 )  http://cavconference.org
	 	 FOUND  0  speakers page(s) 
	 ( 16 / 39 )  http://www.aaai.org/
	 	 FOUND  0  speakers page(s) 
	 ( 17 / 39 )  http://www.aamas-conference.org/
	 	 FOUND  0  speakers page(s) 
	 ( 18 / 39 )  https://www.eurai.org/activities/ECAI_conferences
	 	 FOUND  0  speakers page(s) 
	 ( 19 / 39 )  http://www.ecmlpkdd.org
	 	 FOUND  0  speakers page(s) 
	 ( 20 / 39 )  http://icml.cc
	 	 FOUND  1  speakers page(s) 
	 ( 21 / 39 )  http://www.ijcai.org/
	 	 FOUND  0  speakers page(s) 
	 ( 22 / 39 )  http://nips.cc/
	 	 FOUND  0  speakers page(s) 
	 ( 23 / 39 )  http://www.informatik.uni-trier.de/~ley/db/conf/ruleml/
	 	 FOUND  0  speakers page(s) 
	 ( 24 / 39 )  https://ea2017.inria.fr//index.php
something went wrong please try again
	 	 FOUND  0  speakers page(s) 
	 ( 25 / 39 )  http://www.wcci2016.org/
	 	 FOUND  0  speakers page(s) 
	 ( 26 / 39 )  http://sig.sigevo.org/index.html/tiki-index.php
	 	 FOUND  0  speakers page(

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


]
���j����;����=�q�}�c�����;�9���YQ�B�8Ik�#1p8m�DH�Q@���D0�8���R�D�8v�4NA$j	�S�,�J�P��*�9�f�ܦ�p��ovwvf&lt;�������֑���s��v%�fd9�
��C�7�~O�+Ol�(���75M]`�7"�
�X�M!��������HC������L�	/�����Z9�"��r�~gҙv��sh�5dR��y����1֞Z#��|z��#5�kЕs�4A�W�9P�M��+�p����
�`P8*CG���*%�[k7���^k�m�tAt�lr��8Nrz�Г6E��OKV�z��T���	��ر��������a��GSU���P�n1-�Tp&amp;:��3���K�Vo �q�T ݄J���^�t�t��ǢY*�Z�+h5��k4,�_ӱ���Zn�H�j1%�C&amp;�@B�5�ZD��R˺! a^��T�Kv�
�0�Sȵ�)�w�m���-\c� ���ې�0O�@Q%R�����ތ�t%e�L���n�"**+�cU�*�# +1�S�+���n��������ė�h<hy�_<?�����������x�$����?���'=��g�φx������f>73\�{��pH�y�Oz��e�N7��Ɇ9,��	J���Qy�T�)���)���Y�{��([
�"�d�Q�Lֹ�\i_�r�
��ʇ�`��:�X���DNm44��"�5) !l����Ad�@��%���b�����!�J	�A�?���P����xk{�έfsI\�7�`����T�5P�@4]O�J�����؂vW�脎vv'��^�U��EUMg�"�ˈ��uW���4��xѱ�ѷ��z!b�I	�^�dy�+�sI:I!��xs�=
��^kqG9�a=C��̐�K�B���3|[O��*+�e�������p�H��J�q���`���,��)����Q�!��&amp;�1v2u�҉��y|O�Y6o�&gt;bL�

...

	 	 FOUND  0  speakers page(s) 
	 ( 387 / 844 )  http://www.sigdial.org/workshops/conference19/
	 	 FOUND  0  speakers page(s) 
	 ( 388 / 844 )  http://wollic.org/?p=10320
	 	 FOUND  0  speakers page(s) 
	 ( 389 / 844 )  http://storageconference.us/
	 	 FOUND  0  speakers page(s) 
	 ( 390 / 844 )  http://wollic.org/?p=10927
	 	 FOUND  0  speakers page(s) 
	 ( 391 / 844 )  http://cidrdb.org/cidr2019/index.html
	 	 FOUND  0  speakers page(s) 
	 ( 392 / 844 )  http://wollic.org/?p=9969
	 	 FOUND  0  speakers page(s) 
	 ( 393 / 844 )  http://algo2018.hiit.fi/esa/
	 	 FOUND  0  speakers page(s) 
	 ( 394 / 844 )  http://wollic.org/?p=9921
	 	 FOUND  0  speakers page(s) 
	 ( 395 / 844 )  http://re18.org/calls_dates/callForResearchPapers.html
	 	 FOUND  2  speakers page(s) 
	 ( 396 / 844 )  http://wollic.org/?p=10321
	 	 FOUND  0  speakers page(s) 
	 ( 397 / 844 )  http://icnp18.cs.ucr.edu/
	 	 FOUND  0  speakers page(s) 
	 ( 398 / 844 )  http://wollic.org/?p=7664
	 	 FOUND  0  speakers page(s

	 ( 458 / 844 )  http://wollic.org/?p=9724
	 	 FOUND  0  speakers page(s) 
	 ( 459 / 844 )  http://www.wi-opt.org/
	 	 FOUND  0  speakers page(s) 
	 ( 460 / 844 )  http://wollic.org/?p=9629
	 	 FOUND  0  speakers page(s) 
	 ( 461 / 844 )  http://mdmconferences.org/mdm2018/
	 	 FOUND  0  speakers page(s) 
	 ( 462 / 844 )  http://wollic.org/?p=9927
	 	 FOUND  0  speakers page(s) 
	 ( 463 / 844 )  http://cp2018.a4cp.org/
	 	 FOUND  0  speakers page(s) 
	 ( 464 / 844 )  http://wollic.org/?p=9972
	 	 FOUND  0  speakers page(s) 
	 ( 465 / 844 )  https://avss2018.org/
	 	 FOUND  1  speakers page(s) 
something went wrong please try again
something went wrong please try again
something went wrong please try again
something went wrong please try again
something went wrong please try again
	 ( 466 / 844 )  http://wollic.org/?p=7766
	 	 FOUND  0  speakers page(s) 
	 ( 467 / 844 )  https://2018.jcdl.org/cfp.html
	 	 FOUND  0  speakers page(s) 
	 ( 468 / 844 )  http://wollic.org/?p=10344
	 	 FOUND  

	 	 FOUND  0  speakers page(s) 
	 ( 519 / 844 )  http://wollic.org/?p=10348
	 	 FOUND  0  speakers page(s) 
	 ( 520 / 844 )  https://waset.org/conference/2018/01/singapore/ICSOC
	 	 FOUND  0  speakers page(s) 
	 ( 521 / 844 )  http://wollic.org/?p=9369
	 	 FOUND  0  speakers page(s) 
	 ( 522 / 844 )  http://ieeevr.org/2018/
	 	 FOUND  2  speakers page(s) 
	 ( 523 / 844 )  http://wollic.org/?p=9980
	 	 FOUND  0  speakers page(s) 
	 ( 524 / 844 )  http://2018.ieee-rfid.org/
	 	 FOUND  1  speakers page(s) 
	 ( 525 / 844 )  http://wollic.org/?p=10349
	 	 FOUND  0  speakers page(s) 
	 ( 526 / 844 )  https://www.ares-conference.eu/
	 	 FOUND  4  speakers page(s) 
	 ( 527 / 844 )  http://wollic.org/?p=7817
	 	 FOUND  0  speakers page(s) 
	 ( 528 / 844 )  http://collaboratecom.org/2017/show/home
	 	 FOUND  2  speakers page(s) 
	 ( 529 / 844 )  http://wollic.org/?p=10364
	 	 FOUND  0  speakers page(s) 
	 ( 530 / 844 )  http://www.icmla-conference.org/icmla18/
	 	 FOUND  1  speakers page(s) 
	 (

	 	 FOUND  0  speakers page(s) 
	 ( 563 / 844 )  http://ictai2017.org
	 	 FOUND  0  speakers page(s) 
	 ( 564 / 844 )  http://wollic.org/?p=7483
	 	 FOUND  0  speakers page(s) 
	 ( 565 / 844 )  https://www.eurographics2018.nl/
	 	 FOUND  2  speakers page(s) 
	 ( 566 / 844 )  http://wollic.org/?p=9984
	 	 FOUND  0  speakers page(s) 
	 ( 567 / 844 )  https://www.systor.org/2018/
	 	 FOUND  1  speakers page(s) 
	 ( 568 / 844 )  http://wollic.org/?p=7827
	 	 FOUND  0  speakers page(s) 
	 ( 569 / 844 )  http://ieeevr.org/2018/contribute/papers.html
	 	 FOUND  1  speakers page(s) 
	 ( 570 / 844 )  http://wollic.org/?p=7828
	 	 FOUND  0  speakers page(s) 
	 ( 571 / 844 )  <a href="http://ijcar2018.org/" style="font-size:11px;line-height:11px;margin:0px;padding:0px;" target="_blank">http://ijcar2018.org/</a>
something went wrong please try again
	 	 FOUND  0  speakers page(s) 
	 ( 572 / 844 )  http://wollic.org/?p=7829
	 	 FOUND  0  speakers page(s) 
	 ( 573 / 844 )  http://www.iconip2017.org/

	 	 FOUND  1  speakers page(s) 
	 ( 620 / 844 )  http://wollic.org/?p=7839
	 	 FOUND  0  speakers page(s) 
	 ( 621 / 844 )  http://embeddedandvlsidesignconference.org/
	 	 FOUND  7  speakers page(s) 
	 ( 622 / 844 )  <a href="" style="font-size:11px;line-height:11px;margin:0px;padding:0px;" target="_blank"></a>
something went wrong please try again
	 	 FOUND  0  speakers page(s) 
	 ( 623 / 844 )  http://wollic.org/?p=7840
	 	 FOUND  0  speakers page(s) 
	 ( 624 / 844 )  http://icnc-fskd.guet.cn/icnc_fskd/call.htm
	 	 FOUND  0  speakers page(s) 
	 ( 625 / 844 )  http://wollic.org/?p=7409
	 	 FOUND  0  speakers page(s) 
	 ( 626 / 844 )  http://ppsn2018.dei.uc.pt/
	 	 FOUND  1  speakers page(s) 
	 ( 627 / 844 )  http://wollic.org/?p=7411
	 	 FOUND  0  speakers page(s) 
	 ( 628 / 844 )  https://www.coin-laboratory.com/emo2019
	 	 FOUND  0  speakers page(s) 
	 ( 629 / 844 )  http://wollic.org/?p=6763
	 	 FOUND  0  speakers page(s) 
	 ( 630 / 844 )  http://www.ieee-icsc.org
	 	 FOUND  0  spe